# Stochastic semi-nonparametric envelopment of z variables data (StoNEZD)

   + Author : Sheng Dai (sheng.dai@aalto.fi)
   + Date : April 16, 2020

### Objective: Estimating a log-transformed cost function model with z-variable and technicial inefficiency

In [1]:
# import package pystoned
from pystoned import *

import pandas as pd
import numpy as np

### Import data

In [2]:
# import execl
df = pd.read_excel ('data.xlsx')

In [3]:
# output (total cost)
y  = df['TOTEX']
y  = y.to_numpy()
y  = y.tolist()

In [4]:
# inputs 
x1  = df['Energy']
x1  = np.asmatrix(x1).T
x2  = df['Length']
x2  = np.asmatrix(x2).T
x3  = df['Customers']
x3  = np.asmatrix(x3).T
x   = np.concatenate((x1, x2, x3), axis=1)
x   = x.tolist()

In [5]:
# Z variables
z = df['PerUndGr']
z = z.to_numpy()
z = z.tolist()

### Stage 1: Convex Nonparametric Least Squares estimation

In [6]:
# import the CNLSZ module
crt   = "mult"
func  = "cost"
pps   = "crs"

model = CNLSZ.cnlsz(y, x, z, crt, func, pps)

In [7]:
# slove the model
from pyomo.environ import *
from pyomo.opt import SolverFactory

solver_manager = SolverManagerFactory('neos')
results = solver_manager.solve(model, opt='knitro', tee=True)

In [8]:
# display estimates
model.e.display() 
model.b.display() 

e : residuals
    Size=89, Index=i
    Key : Lower : Value                   : Upper : Fixed : Stale : Domain
      0 :  None :     0.06655983039207396 :  None : False : False :  Reals
      1 :  None :     0.02602719052992247 :  None : False : False :  Reals
      2 :  None :    0.043648066751909576 :  None : False : False :  Reals
      3 :  None :    -0.00886795252472355 :  None : False : False :  Reals
      4 :  None :     0.18923047016780992 :  None : False : False :  Reals
      5 :  None :    -0.11484874779774731 :  None : False : False :  Reals
      6 :  None :   -0.015960384511863827 :  None : False : False :  Reals
      7 :  None :     0.15712234554095753 :  None : False : False :  Reals
      8 :  None :     0.22759422205142152 :  None : False : False :  Reals
      9 :  None :     -0.0871259695102527 :  None : False : False :  Reals
     10 :  None :    0.034452121198105735 :  None : False : False :  Reals
     11 :  None :    -0.07715593832717693 :  None : False : False

In [10]:
# retrive the residuals
val = list(model.e[:].value)
eps = np.asarray(val)

# retrive the beta
ind = list(model.b)
val = list(model.b[:,:].value)
beta= np.asarray([ i + tuple([j]) for i, j in zip(ind, val)]) 

import pandas as pd
beta = pd.DataFrame(beta, columns = ['Name', 'Key', 'Value'])
beta = beta.pivot(index='Name', columns='Key', values='Value')
beta.columns = ['b1', 'b2', 'b3']

### Stage 2: Residuals decomposition

In [11]:
# decomposition by method od moment (MoM)
method      = 'MoM'
TE          = StoNED.stoned(eps, func, method, crt)

In [12]:
# display technicial inefficiency
TE

array([1.03121488, 1.05821989, 1.0463938 , 1.08203532, 0.953615  ,
       1.15770202, 1.08694095, 1.00165531, 0.93055998, 1.13741232,
       1.05254899, 1.13020274, 0.93627356, 0.93209742, 1.11681619,
       1.14838995, 1.00193743, 0.91453345, 1.06479365, 1.14353791,
       1.12467323, 1.28367281, 0.90737425, 1.1524763 , 1.00181278,
       1.06247836, 0.92905189, 1.22067477, 0.9562604 , 1.09164404,
       1.11123754, 1.29828365, 1.09910998, 0.92965215, 1.09167268,
       1.00195347, 1.22598376, 1.20831028, 1.17581079, 1.08475325,
       1.07620146, 1.07165164, 1.0062266 , 1.1493666 , 1.15952405,
       1.1741311 , 1.04900829, 1.00969688, 1.17437576, 1.19141194,
       0.92226044, 1.05319116, 1.00307759, 0.94380991, 1.02918894,
       1.23855822, 1.19505691, 1.10767464, 1.22257961, 1.09886741,
       1.18311496, 1.11828531, 1.10932592, 1.04443526, 0.89711915,
       1.13285104, 1.00370318, 1.12860966, 1.04627088, 1.28544199,
       1.09466009, 1.05638896, 1.178573  , 1.13774757, 1.16607